In [2]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

import torch, random, numpy as np
from transformers import set_seed

def set_all_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    set_seed(seed)
    torch.backends.cudnn.deterministic = True

set_all_seeds(9)

from safetensors import safe_open

In [3]:
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")
messages = [
    {
        "role": "user",
        "content": [
            # {"type": "image"},
            # {"type": "text", "text": "Can you describe the image?"}
            {"type": "text", "text": "What is life?"}
            # {"type": "text", "text": "A real-valued function f defined on the real line is called an even function if f(-t) = f(t) for each real number t. Prove that the set of even functions defined on the real line with the operations of addition and scalar multiplication defined in Example 3 is a vector space."}
        ]
    },
]

In [5]:
model_cuda = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-Instruct",
    torch_dtype=torch.bfloat16,
    # _attn_implementation="flash_attention_2",  # Commented out Flash Attention
    device_map="cuda",
)
model_cuda.eval()

model_cpu = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-Instruct",
    torch_dtype=torch.float32,
    # _attn_implementation="flash_attention_2",  # Commented out Flash Attention
    device_map="cpu",
)
model_cpu.eval();

In [7]:
# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
# inputs = processor(text=prompt, images=[image1], return_tensors="pt")
inputs = processor(text=prompt, return_tensors="pt")

# Generate outputs
with torch.no_grad():
    outputs_cuda = model_cuda.generate(
        **inputs.to("cuda"),
        max_new_tokens=500,
        # repition_penalty=1.1,  # Apply repeat penalty
        output_scores=True,           # Return logits for each generated token
        return_dict_in_generate=True, # Return detailed output object
        do_sample=False,  # Use greedy decoding (highest logit)
    )


# Generate outputs
with torch.no_grad():
    outputs_cpu = model_cpu.generate(
        **inputs.to("cpu"),
        max_new_tokens=500,
        # repition_penalty=1.1,  # Apply repeat penalty
        output_scores=True,           # Return logits for each generated token
        return_dict_in_generate=True, # Return detailed output object
        do_sample=False,  # Use greedy decoding (highest logit)
    )

In [8]:
outputs_cuda.sequences[0], outputs_cpu.sequences[0]

(tensor([    1, 11126,    42,  1812,   314,  1029,    47, 49154,   198,  9519,
          9531,    42,  5330,   314,   253,  1784,   284,  4013,  1909,   338,
           553,   719,   260,  2138,   282, 11150,    28,  3097,    28,   284,
          4955, 13926,   327,  5034,    30,  1814,   624,   768,  2987,  1106,
            28,  1029,   314,   260,  9231,   338, 29792,  2242,  6896,   429,
         47600,  3401,   284,  1603,    29, 23209,  2631,    30, 12532,  6896,
           359,  5181,   282,  2063,    28,   639,    29, 46218, 10751,    28,
         12166,    28,   284,  2426,   288,   260,  1357,    30,   198,   198,
          5212,   253,  3097,  4939,    28,  1029,   314,  1129,  4355,   411,
           260,  4313,   282,   253,  5748,  1297,    28,   527,   314,   260,
          2987,  2723,   282,   511,  2242,  6896,    30, 22189,   359,  1135,
           614,   282,  1461, 41266,   284,  3626,   338,   746,  1592,   288,
          1477,  1895,  3691,   715,   347, 10751,  

In [9]:
processor.tokenizer.decode(outputs_cuda.sequences[0], skip_special_tokens=True)

'User: What is life?\nAssistant: Life is a complex and fascinating concept that has been the subject of philosophical, scientific, and spiritual inquiry for centuries. At its most basic level, life is the characteristic that distinguishes living organisms from inanimate objects and non-living matter. Living organisms are capable of growth, self-sustaining metabolism, reproduction, and response to the environment.\n\nFrom a scientific perspective, life is often defined by the presence of a biological cell, which is the basic unit of all living organisms. Cells are made up of various organelles and structures that work together to perform essential functions such as metabolism, growth, and reproduction.\n\nFrom a philosophical perspective, life is often associated with consciousness, self-awareness, and the ability to experience emotions and suffering. Some philosophers argue that life is a unique and precious gift that should be valued and protected, while others believe that life is a 

In [10]:
processor.tokenizer.decode(outputs_cpu.sequences[0], skip_special_tokens=True)

'User: What is life?\nAssistant: Life is the most precious gift that we have. It is the most valuable thing that we have. It is the most important thing that we have. It is the most beautiful thing that we have. It is the most amazing thing that we have. It is the most wonderful thing that we have. It is the most wonderful thing that we have. It is the most amazing thing that we have. It is the most beautiful thing that we have. It is the most precious gift that we have. It is the most valuable thing that we have. It is the most important thing that we have. It is the most beautiful thing that we have. It is the most amazing thing that we have. It is the most wonderful thing that we have. It is the most wonderful thing that we have. It is the most amazing thing that we have. It is the most beautiful thing that we have. It is the most precious gift that we have. It is the most valuable thing that we have. It is the most important thing that we have. It is the most beautiful thing that w

# PyTorch CUDA/bf16
Life is a complex and fascinating concept that has been the subject of philosophical, scientific, and spiritual inquiry for centuries. At its most basic level, life is the characteristic that distinguishes living organisms from inanimate objects and non-living matter. Living organisms are capable of growth, self-sustaining metabolism, reproduction, and response to the environment.\n\nFrom a scientific perspective, life is often defined by the presence of a biological cell, which is the basic unit of all living organisms. Cells are made up of various organelles and structures that work together to perform essential functions such as metabolism, growth, and reproduction.\n\nFrom a philosophical perspective, life is often associated with consciousness, self-awareness, and the ability to experience emotions and suffering. Some philosophers argue that life is a unique and precious gift that should be valued and protected, while others believe that life is a series of experiences and interactions that ultimately lead to death.\n\nFrom a spiritual perspective, life is often seen as a journey towards enlightenment, self-discovery, and the ultimate goal of achieving a state of union with the divine. Many religious traditions emphasize the importance of living a virtuous and purposeful life, and the goal of achieving a state of enlightenment or nirvana.\n\nIn summary, life is a complex and multifaceted concept that has been explored and understood in many different ways throughout history. It is a fundamental aspect of human existence, and one that is both celebrated and mourned by people of all cultures and religions.


# PyTorch CPU/f32
Life is the most precious gift that we have. It is the most valuable thing that we have. It is the most important thing that we have. It is the most beautiful thing that we have. It is the most amazing thing that we have. It is the most wonderful thing that we have. It is the most wonderful thing that we have. It is the most amazing thing that we have. It is the most beautiful thing that we have. It is the most precious gift that we have. It is the most valuable thing that we have. It is the most important thing that we have. It is the most beautiful thing that we have. It is the most amazing thing that we have. It is the most wonderful thing that we have. It is the most wonderful thing that we have. It is the most amazing thing that we have. It is the most beautiful thing that we have. It is the most precious gift that we have. It is the most valuable thing that we have. It is the most important thing that we have. It is the most beautiful thing that we have. It is the most amazing thing that we have. It is the most wonderful thing that we have. It is the most wonderful thing that we have. It is the most beautiful thing that we have. It is the most precious gift that we have. It is the most valuable thing that we have. It is the most important thing that we have. It is the most beautiful thing that we have. It is the most amazing thing that we have. It is the most wonderful thing that we have. It is the most wonderful thing that we have. It is the most beautiful thing that we have. It is the most precious gift that we have. It is the most valuable thing that we have. It is the most important thing that we have. It is the most beautiful thing that we have. It is the most amazing thing that we have. It is the most wonderful thing that we have. It is the most wonderful thing that we have. It is the most beautiful thing that we have. It is the most precious gift that we have. It is the most valuable thing that we have. It is the most important thing that we have. It is the most beautiful thing that we have. It is the most amazing thing that we have. It is the most wonderful thing that we have. It is the most wonderful thing that we have. It is the most beautiful thing that we have.

# Candle CUDA/bf16
Life is a complex and fascinating phenomenon that has been the subject of study and contemplation for centuries. It is a fundamental aspect of the universe, and it is essential for the existence of all living organisms.

Life is characterized by certain key features that distinguish it from non-living matter. These features include the ability to grow, reproduce, respond to stimuli, and adapt to the environment. Life is also characterized by the presence of energy, which is necessary for all living organisms to function.

The origins of life are a topic of ongoing research, but it is believed that life first emerged on Earth approximately 3.8 billion years ago. The first living organisms were simple, single-celled organisms that were able to perform basic functions such as growth, reproduction, and metabolism. Over time, these organisms evolved into more complex forms, eventually leading to the diversity of life that we see today.

The study of life is a multidisciplinary field that draws on insights from biology, chemistry, physics, and other sciences. Scientists are constantly exploring new ways to understand the fundamental principles of life, and to develop new technologies that can help us to better understand and manipulate living systems.

In conclusion, life is a complex and fascinating phenomenon that is essential for the existence of all living organisms. It is a fundamental aspect of the universe, and it is essential for the continued survival of our planet.

# Candle CPU/f32
 Life is the most precious gift that we have. It is ...